# Imports

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Read Excel

In [2]:
df_exe = pd.read_excel("tcis_with_groupdata.xlsx", index_col=None, dtype=str, na_values=" ", usecols=["Thales P/N on BOM","lien", "PartId","BcnNumber", "PartType", "Part Description (En)", "PartNumber", "OrgName42", "A750Code", "Main Classif Code", "DenomCode", "CtaCode", "TargetPartId", "CIS:AML Status", "CIS:MPN Id"])
df_exe = df_exe.fillna("")
df_exe = df_exe.drop_duplicates(subset=["Thales P/N on BOM", "PartId", "TargetPartId"])

display(HTML(df_exe.to_html()))

,Thales P/N on BOM,lien,PartId,BcnNumber,PartNumber,PartType,Part Description (En),OrgName42,A750Code,Main Classif Code,DenomCode,CtaCode,TargetPartId,CIS:AML Status,CIS:MPN Id
0,91353388,Tar,5041671,99260344,CLS-M4-2 ROHS,GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,THALES GLOBAL SERVICES EPM/BCN,73260,M020206,215,425,,,
1,91353388,Tar,4978391,302-002210,TASUK - 302-002210,UNIT GROUP,BUSH:SELF-CLINCHING CIRCULAR M4,THALES UK LTD. ISR (DMS),73260,M020206,215,425,,,
2,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,5041671,MA,
3,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4803531,MA,
4,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4978391,MA,
5,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,2838295,MA,
7,99230548,Tar*,2752823,99230548,114-60080,GROUP,CON-POWER*FEM*15*PF_15A*AG_CON-2.54MM DIN41612*10.16mm*Pressfit,THALES GLOBAL SERVICES EPM/BCN,44130,E41020201,211,441,,,
12,99230548,sce,2696091,91913397,114-60080,COMMERCIAL,CON-POWER*FEM*15*PF_15A*AG_CON-2.54MM DIN41612*10.16mm*Pressfit,EPT GMBH,44130,E41020201,211,441,2836878,MD,60811
16,91353388,Tar,2838295,1AD001960019,APN - 1AD001960019,UNIT GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,TRSS CODIFICATION ENTITY,73240,M020204,215,425,,,
21,91353388,sce,2781877,,300104263,COMMERCIAL,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,KOENIG VERBINDUNGSTECHNIK GMBH,73940,M020904,786,425,2838295,MA,


## Fill Groups with Group Part IDs

In [3]:
df_exe["TargetPartId"] = np.where((df_exe["PartType"].str.contains("GROUP")) & df_exe["TargetPartId"].eq(""), df_exe["PartId"], df_exe["TargetPartId"])
# df_exe = df_exe.sort_values(by=["Thales P/N on BOM", "TargetPartId", "PartType"], ascending=[True, True, False])
display(HTML(df_exe.to_html()))


,Thales P/N on BOM,lien,PartId,BcnNumber,PartNumber,PartType,Part Description (En),OrgName42,A750Code,Main Classif Code,DenomCode,CtaCode,TargetPartId,CIS:AML Status,CIS:MPN Id
0,91353388,Tar,5041671,99260344,CLS-M4-2 ROHS,GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,THALES GLOBAL SERVICES EPM/BCN,73260,M020206,215,425,5041671,,
1,91353388,Tar,4978391,302-002210,TASUK - 302-002210,UNIT GROUP,BUSH:SELF-CLINCHING CIRCULAR M4,THALES UK LTD. ISR (DMS),73260,M020206,215,425,4978391,,
2,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,5041671,MA,
3,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4803531,MA,
4,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4978391,MA,
5,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,2838295,MA,
7,99230548,Tar*,2752823,99230548,114-60080,GROUP,CON-POWER*FEM*15*PF_15A*AG_CON-2.54MM DIN41612*10.16mm*Pressfit,THALES GLOBAL SERVICES EPM/BCN,44130,E41020201,211,441,2752823,,
12,99230548,sce,2696091,91913397,114-60080,COMMERCIAL,CON-POWER*FEM*15*PF_15A*AG_CON-2.54MM DIN41612*10.16mm*Pressfit,EPT GMBH,44130,E41020201,211,441,2836878,MD,60811
16,91353388,Tar,2838295,1AD001960019,APN - 1AD001960019,UNIT GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,TRSS CODIFICATION ENTITY,73240,M020204,215,425,2838295,,
21,91353388,sce,2781877,,300104263,COMMERCIAL,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,KOENIG VERBINDUNGSTECHNIK GMBH,73940,M020904,786,425,2838295,MA,


## Split into Groups and Parse

In [4]:
def clean(group):
    if (any(group["PartType"].str.contains("GROUP"))):
        group = group[group["TargetPartId"].isin(group["PartId"])]
        group = group.sort_values(by=["Thales P/N on BOM", "TargetPartId", "PartType"], ascending=[True, True, False])
#         display(HTML(group.to_html()))
    else:
        print("Floating Sources")
    return group

df_exe = df_exe.groupby("Thales P/N on BOM", as_index=False).apply(func=clean)
df_exe = df_exe.reset_index(drop=True)

In [5]:
display(HTML(df_exe.to_html()))

,Thales P/N on BOM,lien,PartId,BcnNumber,PartNumber,PartType,Part Description (En),OrgName42,A750Code,Main Classif Code,DenomCode,CtaCode,TargetPartId,CIS:AML Status,CIS:MPN Id
0,91353388,Tar,2838295,1AD001960019,APN - 1AD001960019,UNIT GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,TRSS CODIFICATION ENTITY,73240,M020204,215,425,2838295,,
1,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,2838295,MA,
2,91353388,sce,2781877,,300104263,COMMERCIAL,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,KOENIG VERBINDUNGSTECHNIK GMBH,73940,M020904,786,425,2838295,MA,
3,91353388,Tar,4803531,XS84130,GTSUK - XS84130,UNIT GROUP,RIVET BUSH M4,THALES GROUND TRANSPORTATION SYSTEMS LTD,73260,M020206,215,425,4803531,,
4,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4803531,MA,
5,91353388,Tar,4978391,302-002210,TASUK - 302-002210,UNIT GROUP,BUSH:SELF-CLINCHING CIRCULAR M4,THALES UK LTD. ISR (DMS),73260,M020206,215,425,4978391,,
6,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4978391,MA,
7,91353388,Tar,5041671,99260344,CLS-M4-2 ROHS,GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,THALES GLOBAL SERVICES EPM/BCN,73260,M020206,215,425,5041671,,
8,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,5041671,MA,
9,99230548,Tar*,2752823,99230548,114-60080,GROUP,CON-POWER*FEM*15*PF_15A*AG_CON-2.54MM DIN41612*10.16mm*Pressfit,THALES GLOBAL SERVICES EPM/BCN,44130,E41020201,211,441,2752823,,


## Rename Dataframe Columns to Prep for Integration with Master TCIS AMPL Dataframe

In [6]:
df_exe.columns = ["ThalesPNBOM", "lien", "ThalesPID", "BCNUpdated", "MPNUpdated", "PartType", "DescrUpdated", "MFRUpdated", "A750", "ClassificationCode", "DenominationCode", "CTACode", "GroupPID", "AMLUpdated", "HERSUpdated"]
df_exe["Live/Local"] = "Live"

display(HTML(df_exe.to_html()))

,ThalesPNBOM,lien,ThalesPID,BCNUpdated,MPNUpdated,PartType,DescrUpdated,MFRUpdated,A750,ClassificationCode,DenominationCode,CTACode,GroupBCN,AMLUpdated,HERSUpdated,Live/Local
0,91353388,Tar,2838295,1AD001960019,APN - 1AD001960019,UNIT GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,TRSS CODIFICATION ENTITY,73240,M020204,215,425,2838295,,,Live
1,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,2838295,MA,,Live
2,91353388,sce,2781877,,300104263,COMMERCIAL,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,KOENIG VERBINDUNGSTECHNIK GMBH,73940,M020904,786,425,2838295,MA,,Live
3,91353388,Tar,4803531,XS84130,GTSUK - XS84130,UNIT GROUP,RIVET BUSH M4,THALES GROUND TRANSPORTATION SYSTEMS LTD,73260,M020206,215,425,4803531,,,Live
4,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4803531,MA,,Live
5,91353388,Tar,4978391,302-002210,TASUK - 302-002210,UNIT GROUP,BUSH:SELF-CLINCHING CIRCULAR M4,THALES UK LTD. ISR (DMS),73260,M020206,215,425,4978391,,,Live
6,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,4978391,MA,,Live
7,91353388,Tar,5041671,99260344,CLS-M4-2 ROHS,GROUP,NUT*RND*INSERT_M4*3.37MM_SS*PASSIV,THALES GLOBAL SERVICES EPM/BCN,73260,M020206,215,425,5041671,,,Live
8,91353388,sce*,1238373,91353388,CLS-M4-2,COMMERCIAL,WASHER THREAD=4M; STAINLESS STEEL,PENN ENGINEERING FASTENING TECHNOLOGIES,73260,M020206,215,425,5041671,MA,,Live
9,99230548,Tar*,2752823,99230548,114-60080,GROUP,CON-POWER*FEM*15*PF_15A*AG_CON-2.54MM DIN41612*10.16mm*Pressfit,THALES GLOBAL SERVICES EPM/BCN,44130,E41020201,211,441,2752823,,,Live
